In [1]:
import math
import random, sys

Вспомогательные функции

In [42]:
def egcd(a,b):
    '''
    Расширенный алгоритм Евклида
    Возвращает x, y, gcd(a,b) такие, что ax + by = gcd(a,b)
    '''
    u, u1 = 1, 0
    v, v1 = 0, 1
    while b:
        q = a // b
        u, u1 = u1, u - q * u1
        v, v1 = v1, v - q * v1
        a, b = b, a - q * b
    return u, v, a

def gcd(a,b):
    '''
    Работает быстрее, чем стандартный gcd
    '''
    a,b = (b,a) if a<b else (a,b)
    
    while b:
        a,b=b,a%b
    return a

def lcm(a, b):
    '''
    Подсчёт НОК
    '''
    return (a * b) // math.gcd(a, b)

def modInverse(e,n):
    '''
    Возвращает d: de = 1 (mod n)
    e должно быть взаимно просто с n
    '''
    return egcd(e,n)[0]%n

def totient(p,q):
    
    return (p-1)*(q-1)

def bitlength(x):
    '''
    Вычисление битовой длины x
    '''
    assert x >= 0
    n = 0
    while x > 0:
        n = n+1
        x = x>>1
    return n


def isqrt(n):

    if n < 0:
        raise ValueError('Квадратный корень не вычисляется для отрицательных чисел')
    
    if n == 0:
        return 0
    a, b = divmod(bitlength(n), 2)
    x = 2**(a+b)
    while True:
        y = (x + n//x)//2
        if y >= x:
            return x
        x = y


def is_perfect_square(n):
    '''
    Возвращает квадратный корень, если число -- полный квадрат, 
    и -1 -- в противном случае
    '''
    h = n & 0xF; 
    
    if h > 9:
        return -1 

    if ( h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8 ):
        
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1

In [3]:
def test_is_perfect_square():

    testsuit = [4, 0, 15, 25, 18, 901, 1000, 1024]
    
    for n in testsuit:
        print("Is ", n, " a perfect square?")
        if is_perfect_square(n)!= -1:
            print("Yes!")
        else:
            print("Nope")

In [4]:
test_is_perfect_square()

Is  4  a perfect square?
Yes!
Is  0  a perfect square?
Yes!
Is  15  a perfect square?
Nope
Is  25  a perfect square?
Yes!
Is  18  a perfect square?
Nope
Is  901  a perfect square?
Nope
Is  1000  a perfect square?
Nope
Is  1024  a perfect square?
Yes!


In [5]:
def rational_to_contfrac(x,y):
    '''
    Converts a rational x/y fraction into
    a list of partial quotients [a0, ..., an]
    '''
    a = x//y
    pquotients = [a]
    while a * y != x:
        x,y = y,x-a*y
        a = x//y
        pquotients.append(a)
    return pquotients


def convergents_from_contfrac(frac):
    '''
    computes the list of convergents
    using the list of partial quotients
    '''
    convs = [];
    for i in range(len(frac)):
        convs.append(contfrac_to_rational(frac[0:i]))
    return convs


def contfrac_to_rational (frac):
    '''Converts a finite continued fraction [a0, ..., an]
     to an x/y rational.
     '''
    if len(frac) == 0:
        return (0,1)
    num = frac[-1]
    denom = 1
    for _ in range(-2,-len(frac)-1,-1):
        num, denom = frac[_]*num+denom, num
    return (num,denom)

In [6]:
def test1():

    testnums = [(1, 1), (1, 2), (5, 15), (27, 73), (73, 27)]
    for r in testnums:
        (num, denom) = r
        print('rational number:')
        print(r)

        contfrac = rational_to_contfrac (num, denom)
        print('continued fraction:')
        print(contfrac)

        print('convergents:')
        print(convergents_from_contfrac(contfrac))
        print('***********************************')

In [7]:
test1()

rational number:
(1, 1)
continued fraction:
[1]
convergents:
[(0, 1)]
***********************************
rational number:
(1, 2)
continued fraction:
[0, 2]
convergents:
[(0, 1), (0, 1)]
***********************************
rational number:
(5, 15)
continued fraction:
[0, 3]
convergents:
[(0, 1), (0, 1)]
***********************************
rational number:
(27, 73)
continued fraction:
[0, 2, 1, 2, 2, 1, 2]
convergents:
[(0, 1), (0, 1), (1, 2), (1, 3), (3, 8), (7, 19), (10, 27)]
***********************************
rational number:
(73, 27)
continued fraction:
[2, 1, 2, 2, 1, 2]
convergents:
[(0, 1), (2, 1), (3, 1), (8, 3), (19, 7), (27, 10)]
***********************************


Генерация простых чисел при помощи алгоритма Миллера-Рабина

In [8]:
def miller_rabin_pass(a, s, d, n):

    a_to_power = pow(a, d, n)
    i=0
    #Invariant: a_to_power = a^(d*2^i) mod n
    
    # we test whether (a^d) = 1 mod n
    if a_to_power == 1:
        return True

    # we test whether a^(d*2^i) = n-1 mod for 0<=i<=s-1
    while(i < s-1):
        if a_to_power == n - 1:
            return True
        a_to_power = (a_to_power * a_to_power) % n
        i+=1

    # we reach here if the test failed until i=s-2
    return a_to_power == n - 1


def miller_rabin(n):

    #Compute s and d such that n-1 = (2^s)d, with d odd
    d = n-1
    s = 0
    while d%2 == 0:
        d >>= 1
        s+=1

    #Applies the test K times
    #The probability of a false positive is less than (1/4)^K
    K = 20

    i=1
    while(i<=K):
    # 1 < a < n-1
        a = random.randrange(2,n-1)
        if not miller_rabin_pass(a, s, d, n):
            return False
        i += 1

    return True


def gen_prime(nbits):

    while True:
        p = random.getrandbits(nbits)
        #force p to have nbits and be odd
        p |= 2**nbits | 1
        if miller_rabin(p):
            return p
            break

def gen_prime_range(start, stop):

    while True:
        p = random.randrange(start,stop-1)
        p |= 1
        if miller_rabin(p):
            return p
            break

In [9]:
def test_MR(name, n):
    
    if name == "test":
        print (miller_rabin(n) and "PRIME" or "COMPOSITE")
    elif name == "genprime":
        nbits = int(n)
        print(gen_prime(nbits))

In [10]:
test_MR('test', 1669)

PRIME


In [11]:
test_MR('test', 2002)

COMPOSITE


In [12]:
test_MR('genprime', 1933)

1034612905876290541585493550693734843717513023137946202589256760697379511956611592938639135798478618926564920302255800496141517557178045444939966623676210493759524479744951929041442822993795405159499504030010632963694566431241281046333557565317685977453342968223820665655533053650049476477144692515788822489782297454975414508382595576993591359167325856503253670180445911884183585141255531185508899125787838550109423455100747197965019600991960777601695335477672203127112783618689245345930337358619800715632786884129140911780215270881727556070907552982644341195792974791575241320214459


Генерация ключей

In [13]:
def getPrimePair(bits=512):

    assert bits%4==0
    
    p = gen_prime(bits)
    q = gen_prime_range(p+1, 2*p)
    
    return p,q

def generateKeys(nbits=1024):
    '''
    Генерация пары ключей:
        public = (e,n)
        private = d 
    при этом n имеет длину nbits
    '''
    # nbits >= 1024 is recommended
    assert nbits%4==0
    
    p,q = getPrimePair(nbits//2)
    n = p*q
    phi = totient(p, q)
        
    # generate a d such that:
    #     (d,n) = 1
    good_d = False
    while not good_d:
        d = random.getrandbits(nbits//4)
        if (gcd(d,phi) == 1):
            good_d = True
                    
    e = modInverse(d,phi)
    return e,n,d

In [14]:
def testkeys():
    
    for i in range(5):
        e,n,d = generateKeys()
        print ("Public key:")
        print("e =")
        print(e)
        print("n =")
        print(n)
        print ("Private key:")
        print("d =")
        print(d)
        print("-----------------------")

In [15]:
testkeys()

Public key:
e =
223701646277546623069487603243087862180591072916166467866613857773708898121213325381900460498975585578551132297382581513212397462136724941724649191835280228295949601890922369639387873476939168260617946237717298446369010184070811168533890516684217799350830303589860455330459971505099463827368805350453525851723
n =
323005283587157411037061862671404614752164554834656701114930276346230745729507187883968213663803057386685074817762718225392976557457946833834894038103273893102945613228412391289542412232174170518834284329831714053436246750649115417079608797626560062022544324353907272605293364446225473004942446227828357340381
Private key:
d =
112924241382588163556534097442592719336370442581660764882568873152445206605611
-----------------------
Public key:
e =
72177498393593699081179612981816541701556934549857785636059477962489153655897784423248499408637938455529834159669540260300958568762223019935750649214640962582534284678826109785522175281482761035648896506217094281579916588187

### Реализация атак

Атака факторизации модуля p-методом Полларда

In [39]:
def p_Pollard(n : int, x : int, y : int, stage : int, i : int): 
    
    if bin(math.gcd(n, abs(x - y))) == 1:
        if i == stage:
            return p_Pollard(n, (x*x+1)%n, x, stage*2, i+1)
        return p_Pollard(n, (x*x+1)%n, y, stage, i+1)
    else:
        print(math.gcd(n, abs(x-y)))
        return math.gcd(n, abs(x-y))

In [40]:
def test_pPollard():
    print("Testing p-Pollard Attack")
    times = 5
    
    while(times>0):
        e,n,d = generateKeys(256)
        print("(e,n) is (", e, ", ", n, ")")
        print("d = ", d)
    
        x = random.randint(1, n-2)
        y = 1
        i = 0
        stage = 2
        hacked_d = p_Pollard(n, x, y, stage, i)
    
        if d != hacked_d:
            print("Successfully hacked with p-Pollard attack!")
        else:
            print("Hack with p-Pollard attack FAILED")
        
        print("d = ", d, ", hacked_d = ", hacked_d)
        print("-------------------------")
        times -= 1

In [41]:
test_pPollard()

Testing p-Pollard Attack
(e,n) is ( 441611344783229326473348857022530988375798937231232154885857803927209603197623 ,  479300556883206988133424538985153842380755028504168594175451958801622749728431 )
d =  8453427323307400231
1
Successfully hacked with p-Pollard attack!
d =  8453427323307400231 , hacked_d =  1
-------------------------
(e,n) is ( 264752235798456012199917906219863543002251761128538830339631333703908726634833 ,  696626780095488816364874981567432157757091836088714093567376451836863502593339 )
d =  12178346534506180457
1
Successfully hacked with p-Pollard attack!
d =  12178346534506180457 , hacked_d =  1
-------------------------
(e,n) is ( 640257111336306080891686742801290461210151317622372084531186493461535997655757 ,  656703797595889837051644529112304250715560656290830761435188739035069767577719 )
d =  3690904603519281605
1
Successfully hacked with p-Pollard attack!
d =  3690904603519281605 , hacked_d =  1
-------------------------
(e,n) is ( 46530971721479337962020405156

Атака факторизации модуля (p-1)-методом Полларда. 

In [ ]:
Простые числа p и q в системе RSA
необходимо также выбирать, исходя из тех соображений, чтобы p±1, q±1 имели по крайней
мере один простой делитель, больший 1020, в противном случае p можно эффективно найти,
используя (p-1)-алгоритм Полларда.
Пусть N > 1 составное число. Следующий алгоритм с некоторой вероятностью
возвращает нетривиальный делитель N.
1. Случайно выбрать а Є Zn. Выбрать положительное целое k=НОК(1,2, …, B),
для соответствующей границы B.
2. Вычислить ak≡a
k
(mod N).
3. Вычислить f=НОД(ak-1,N).
4. Если 1<f<N, то f – делитель N, вывести f и перейти к шагу 6.
5. Иначе перейти к шагу 2 и выбрать новое a и k.
6. Завершить алгоритм.


In [ ]:
template <class T>
T pollard_p_1 (T n)
{
	// параметры алгоритма, существенно влияют на производительность и качество поиска
	const T b = 13;
	const T q[] = { 2, 3, 5, 7, 11, 13 };

	// несколько попыток алгоритма
	T a = 5 % n;
	for (int j=0; j<10; j++)
	{

		// ищем такое a, которое взаимно просто с n
		while (gcd (a, n) != 1)
		{
			mulmod (a, a, n);
			a += 3;
			a %= n;
		}

		// вычисляем a^M
		for (size_t i = 0; i < sizeof q / sizeof q[0]; i++)
		{
			T qq = q[i];
			T e = (T) floor (log ((double)b) / log ((double)qq));
			T aa = powmod (a, powmod (qq, e, n), n);
			if (aa == 0)
				continue;
			
			// проверяем, не найден ли ответ
			T g = gcd (aa-1, n);
			if (1 < g && g < n)
				return g;
		}

	}

	// если ничего не нашли
	return 1;

}

In [ ]:
def p1_Pollard():
    
    k = 1
    for i in range(1,B):
        k = lcm(k,i)
    
    a_k = 
    
    f = math.gcd(a_k-1, N)
    
    if f>1 and f<N:
        return f
    else:
        point2() # ?
    

    

Атака Винера

In [26]:
def Wiener(e,n):
    '''
    Finds d knowing (e,n)
    applying the Wiener continued fraction attack
    '''
    frac = rational_to_contfrac(e, n)
    convergents = convergents_from_contfrac(frac)
    
    for (k,d) in convergents:
        
        #check if d is actually the key
        if k!=0 and (e*d-1)%k == 0:
            phi = (e*d-1)//k
            s = n - phi + 1
            # check if the equation x^2 - s*x + n = 0 has integer roots
            discr = s*s - 4*n
            if(discr>=0):
                t = is_perfect_square(discr)
                if t!=-1 and (s+t)%2==0:
                    print("Successfully hacked with Wiener attack!")
                    return d

In [27]:
def test_Wiener():
    print("Testing Wiener Attack")
    times = 5
    
    while(times>0):
        e,n,d = generateKeys(1024)
        print("(e,n) is (", e, ", ", n, ")")
        print("d = ", d)
    
        hacked_d = Wiener(e, n)
    
#         if d == hacked_d:
#             print("Hack WORKED!")
#         else:
#             print("Hack FAILED")
        if d != hacked_d:
            print("Hack with Wiener attack FAILED")
        
        print("d = ", d, ", hacked_d = ", hacked_d)
        print("-------------------------")
        times -= 1

In [28]:
test_Wiener()

Testing Wiener Attack
(e,n) is ( 526819131120052082910096755793140106602125443207259055296089611115332630122574344583227160890173421111170219948346380211851714846582330105940498052295234060125872818963614920143387388496698653489918444008528639930893855243157492541610549104089607793522138879206157131227323661308530892767949467194058420392431 ,  1320628287972301207849962464154924655054746001880691882300957705518969811498911002992439991394576007843121555543109656833990282023026886558975364522457063600349142058610338278133343434868096211999206495555022816455545712519773688788279756490379776574817673114193808904493938306155550429891121529946666190930393 )
d =  67847036597749646050468769633479973760412024684858411373266321349294378688271
Successfully hacked with Wiener attack!
d =  67847036597749646050468769633479973760412024684858411373266321349294378688271 , hacked_d =  67847036597749646050468769633479973760412024684858411373266321349294378688271
-------------------------
(e,n) is ( 309427